In [ ]:
import pandas as pd
train=pd.read_csv('../input/rte-forecast-energy-consumption-in-french-areas/train.csv')
train.head()

In [ ]:
train.info()

In [ ]:
weather=pd.read_csv('../input/rte-forecast-energy-consumption-in-french-areas/weather_power_nasa.csv')
weather.head()

In [ ]:
weather.info()

In [ ]:
train=pd.merge(train, weather, on=['metropolitan_area_code','date'], how='left')
train['metropolitan_area_code']=train['metropolitan_area_code'].replace({0:'[48.856614, 2.3522219]',
 1:'[45.764043, 4.835659]',
 2:'[43.296482, 5.36978]',
 3:'[44.8378, -0.57918]',
 4:'[50.62925, 3.057256]',
 5:'[43.7101728, 7.2619532]',
 6:'[47.218371, -1.553621]',
 7:'[48.5734053, 7.7521113]',
 8:'[48.117266, -1.6777926]',
 9:'[45.188529, 5.724524]',
 10:'[49.443232, 1.099971]',
 11:'[43.610769, 3.876716]',
 12:'[45.439695, 4.3871779]',
 13:'[45.777222, 3.087025]',
 14:'[48.692054, 6.184417]',
 15:'[47.322047, 5.04148]',
 16:'[48.390394, -4.486076]'})
train['metropolitan_area_code']=train['metropolitan_area_code'].str.replace('[','')
train['metropolitan_area_code']=train['metropolitan_area_code'].str.replace(']','')
train[['latitude','longitude']] = train['metropolitan_area_code'].str.split(',',expand=True)
train=train.drop(['metropolitan_area_code'],axis=1)
train.head()

In [ ]:
train.info()

In [ ]:
train.isnull().sum()*100/len(train)

In [ ]:
train.columns

In [ ]:
train[['T2M', 'T2MDEW',
       'T2MWET', 'QV2M', 'RH2M', 'PRECTOTCORR']] = train[['T2M', 'T2MDEW',
       'T2MWET', 'QV2M', 'RH2M', 'PRECTOTCORR']].ffill()
train.info()

In [ ]:
train.head(25)

In [ ]:
train[['T2M', 'T2MDEW',
       'T2MWET', 'QV2M', 'RH2M', 'PRECTOTCORR']] = train[['T2M', 'T2MDEW',
       'T2MWET', 'QV2M', 'RH2M', 'PRECTOTCORR']].bfill()
train.info()

In [ ]:
train.head()

In [ ]:
train.date=pd.to_datetime(train.date, infer_datetime_format=True)
train.info()

In [ ]:
!pip install fast_ml
!pip install lazypredict

In [ ]:
from fast_ml.feature_engineering import FeatureEngineering_DateTime
dt_fe = FeatureEngineering_DateTime()
dt_fe.fit(train, datetime_variables=['date'])
train = dt_fe.transform(train)
train.head()

In [ ]:
nunique_train=train.nunique().reset_index()
remove_col=nunique_train[(nunique_train[0]==len(train)) | (nunique_train[0]==0) | (nunique_train[0]==1) ]['index'].tolist()
remove_col

In [ ]:
pd.set_option('display.max_columns', None)
train=train.drop(remove_col,axis=1)
train.head()

In [ ]:
train=train.drop(['date','date:time'],axis=1)
train.head()

In [ ]:
train['date:is_weekend']=train['date:is_weekend'].replace({0:False,1:True})
train.head()

In [ ]:
train.info()

In [ ]:
train['date:day_part'].unique()

In [ ]:
train['date:day_part'].value_counts()

In [ ]:
import numpy as np
train['date:day_part'] = train['date:day_part'].fillna(value=np.nan)
train['date:day_part'].unique()

In [ ]:
round(train.isnull().sum()*100/len(train),2).sort_values(ascending=False)

In [ ]:
train[train.isnull().any(axis=1)]

In [ ]:
train['date:day_part'] = train['date:day_part'].fillna(value='midnight')
round(train.isnull().sum()*100/len(train),2).sort_values(ascending=False)[:5]

In [ ]:
train['date:day_part']=train['date:day_part'].replace({'midnight':0,'dawn':1, 'early morning':2, 'late morning':3, 'noon':4,
                                                       'afternoon':5,'evening':6, 'night':7 })
      
train['date:day_part'].value_counts()

In [ ]:
import re
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
train.head()

In [ ]:
train=train.sample(n=30000)
train.shape

In [ ]:
y = train.pop('energy_consumption')
X = train

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [ ]:
from lazypredict.Supervised import LazyRegressor

In [ ]:
clf = LazyRegressor(verbose=0)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models